In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [3]:
num_epochs = 10
batch_size = 64
learning_rate = 1e-3

In [ ]:
device = torch.device('xpu' if torch.xpu.is_available() else 'cpu')
print(f"Usando o dispositivo: {device}")

Usando o dispositivo: cpu


In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

In [6]:
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
model = FashionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        outputs = model(data)
        loss = criterion(outputs, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f'Época de Treino: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tPerda: {loss.item():.6f}')


In [9]:
def evaluate(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            test_loss += criterion(outputs, target).item()
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        print(f'\nConjunto de Teste: Perda Média: {test_loss:.4f}, Acurácia: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [ ]:
for epoch in range(num_epochs):
    train(model, device, train_loader, criterion, optimizer, epoch)
    evaluate(model, device, test_loader)

Época de Treino: 1 [6336/60000 (11%)]	Perda: 0.402969
Época de Treino: 1 [12736/60000 (21%)]	Perda: 0.539141
Época de Treino: 1 [19136/60000 (32%)]	Perda: 0.457055
Época de Treino: 1 [25536/60000 (43%)]	Perda: 0.285695
Época de Treino: 1 [31936/60000 (53%)]	Perda: 0.651266
Época de Treino: 1 [38336/60000 (64%)]	Perda: 0.411316
Época de Treino: 1 [44736/60000 (75%)]	Perda: 0.440702
Época de Treino: 1 [51136/60000 (85%)]	Perda: 0.459366
Época de Treino: 1 [57536/60000 (96%)]	Perda: 0.306651

Conjunto de Teste: Perda Média: 0.0053, Acurácia: 8797/10000 (88%)

Época de Treino: 2 [6336/60000 (11%)]	Perda: 0.237895
Época de Treino: 2 [12736/60000 (21%)]	Perda: 0.372549
Época de Treino: 2 [19136/60000 (32%)]	Perda: 0.413690
Época de Treino: 2 [25536/60000 (43%)]	Perda: 0.215981
Época de Treino: 2 [31936/60000 (53%)]	Perda: 0.343069
Época de Treino: 2 [38336/60000 (64%)]	Perda: 0.417583
Época de Treino: 2 [44736/60000 (75%)]	Perda: 0.384830
Época de Treino: 2 [51136/60000 (85%)]	Perda: 0.25277